In [10]:
import numpy as np
import pandas as pd
import yfinance as yf
import pygad

In [11]:
stocks = ['ITUB4.SA', 'MGLU3.SA', 'RENT3.SA', 'PETR4.SA', 'WEGE3.SA', 'ELET6.SA', 'RADL3.SA']

data = yf.download(stocks, start='2021-01-01', end='2022-12-31')['Adj Close']

data

[*********************100%***********************]  7 of 7 completed


,ELET6.SA,ITUB4.SA,MGLU3.SA,PETR4.SA,RADL3.SA,RENT3.SA,WEGE3.SA
Date,,,,,,,
2021-01-04,30.883387,27.964464,25.181740,11.470779,23.486471,65.240555,35.927433
2021-01-05,29.996132,27.783470,24.742058,11.919136,23.420790,64.855530,38.132568
2021-01-06,29.825504,28.597967,23.442999,11.942941,22.876553,61.120911,39.143669
2021-01-07,29.262438,29.711128,23.143217,12.300040,22.585672,64.018135,40.761421
2021-01-08,30.115570,29.702068,23.822723,12.347651,23.242502,68.339920,43.226547
...,...,...,...,...,...,...,...
2022-12-23,42.900005,24.574018,2.680000,20.983213,22.873898,55.563160,38.168095
2022-12-26,42.582867,24.086246,2.660000,20.832857,22.749372,54.043736,37.860760
2022-12-27,41.621845,23.949667,2.520000,20.974861,22.644007,52.027775,37.682316


In [12]:
returns = data.pct_change().dropna()
returns

,ELET6.SA,ITUB4.SA,MGLU3.SA,PETR4.SA,RADL3.SA,RENT3.SA,WEGE3.SA
Date,,,,,,,
2021-01-05,-0.028729,-0.006472,-0.017460,0.039087,-0.002797,-0.005902,0.061377
2021-01-06,-0.005688,0.029316,-0.052504,0.001997,-0.023237,-0.057584,0.026515
2021-01-07,-0.018879,0.038924,-0.012788,0.029900,-0.012715,0.047402,0.041329
2021-01-08,0.029155,-0.000305,0.029361,0.003871,0.029082,0.067509,0.060477
2021-01-11,-0.021813,-0.022547,-0.014681,-0.008355,-0.005248,-0.019437,-0.030630
...,...,...,...,...,...,...,...
2022-12-23,0.015931,0.015726,0.030769,0.047103,0.013152,0.054665,-0.011299
2022-12-26,-0.007393,-0.019849,-0.007463,-0.007166,-0.005444,-0.027346,-0.008052
2022-12-27,-0.022568,-0.005670,-0.052632,0.006816,-0.004632,-0.037302,-0.004713


In [30]:
def portfolio_returns(solution):
    port_returns = 0
    for i in range(len(solution)):
        port_returns += returns.iloc[:, i] * solution[i]
    return port_returns

def cvm(solution):
    """
    Carteira de variância mínima
    """
    port_returns = portfolio_returns(solution)
    return np.std(port_returns)

def constraints(solution):
    bound = (sum(solution) - 1)**2
    return bound

def fitness_function(ga_instance, solution, solution_idx):
    return (cvm(solution) + 100*constraints(solution))


In [31]:
ga = pygad.GA(num_generations=50, num_parents_mating=10,
              fitness_func=fitness_function, sol_per_pop=20,
              gene_type=np.float32, mutation_percent_genes=10,
              gene_space=bounds, num_genes=num_stocks)

ga.run()

c:\Users\Yamac\AppData\Local\Programs\Python\Python310\lib\site-packages\pygad\pygad.py:738: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


### Teste 2

In [57]:
def portfolio_generate(df):
    portfolio = df.copy()
    portfolio['Total'] = portfolio.sum()
    portfolio['Total'] = portfolio['Total']  / 100
    portfolio['Total'] = portfolio['Total'] + 1
    portfolio = portfolio[['Total']]
    return portfolio

def portfolio_return(portfolio):
    return np.prod(portfolio['Total']) -1

def fitness_function(ga_instance, solution, solution_idx):
    portfolio = portfolio_generate(returns)
    ret = portfolio_return(portfolio)
    return ret

In [65]:
ga_instance = pygad.GA(num_generations=30, num_parents_mating=50,
                       fitness_func=fitness_function, sol_per_pop=90,
                       init_range_low=0, init_range_high=1,
                       parent_selection_type='sss', keep_parents=30,
                       crossover_type='single_point', mutation_percent_genes=10,
                       random_seed=42, num_genes=7)

ga_instance.run() 

c:\Users\Yamac\AppData\Local\Programs\Python\Python310\lib\site-packages\pygad\pygad.py:738: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


In [66]:
ga_instance.best_solution()

(array([6.31302552, 3.92887282, 5.22131094, 5.60241688, 5.04461411,
        5.74429642, 6.69531025]),
 140999.10240085586,
 0)

### Teste 3

In [88]:
teste = [21,32,43,54,65,76,87]

In [89]:
def constraint(solution):
    bounduary_constraint = (sum(solution) - 1)**2
    return bounduary_constraint

def fitness_function(ga_intance, solution, solution_idx):
    fitness = np.dot(teste, solution)
    return fitness -100*constraint(solution)

In [96]:
ga_instance = pygad.GA(num_generations=30, num_parents_mating=50,
                       fitness_func=fitness_function, sol_per_pop=90,
                       init_range_low=0, init_range_high=1,
                       parent_selection_type='sss', keep_parents=30,
                       crossover_type='single_point', mutation_percent_genes=10,
                       random_seed=42, num_genes=7, gene_space=gene_space)

ga_instance.run() 

c:\Users\Yamac\AppData\Local\Programs\Python\Python310\lib\site-packages\pygad\pygad.py:738: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


In [99]:
ga_instance.best_solution()

(array([0.03673194, 0.00977085, 0.03142919, 0.01009061, 0.11573761,
        0.14032452, 0.9623102 ]),
 95.50119723987602,
 0)

In [100]:
np.sum([0.03673194, 0.00977085, 0.03142919, 0.01009061, 0.11573761, 0.14032452, 0.9623102])

1.30639492

### Teste 4 - DEAP

In [106]:
from deap import algorithms, base, creator, tools
import random

In [103]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [104]:
def evaluate_portfolio(individual):
    # Calculate returns and risk based on individual weights
    returns = calculate_returns(individual)
    risk = calculate_risk(individual)

    # Return a tuple of the fitness values (returns, -risk as it's a maximization problem)
    return returns, -risk

In [107]:
toolbox = base.Toolbox()
toolbox.register("individual", tools.initRepeat, creator.Individual, random.uniform, 0, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_portfolio)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)